<a href="https://colab.research.google.com/github/AmorimFernandaS/desafiodados/blob/main/Desafio_Solvimm_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
#Intalando pacote Pyspark

!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
!pip install -q findspark

In [101]:
import findspark
findspark.init()

In [102]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f


#abrindo sessão spark

from pyspark.sql import SparkSession
spark= SparkSession.builder\
       .master('local')\
       .appName('DesafioSolvimm')\
       .getOrCreate()

In [103]:
#Importando arq. movies.csv

df1=spark.read.options(inferSchema='True',delimiter=';',columns=True).csv('/content/drive/MyDrive/movies.csv')

#Nomeando Tabelas
filmes=df1.withColumnRenamed('_c0','Movie_Id').withColumnRenamed('_c1','titulo_ano')

In [104]:
#Vizualizando DF

filmes.show(truncate=False)

+--------+-----------------------------------------------------------------------------------+
|Movie_Id|titulo_ano                                                                         |
+--------+-----------------------------------------------------------------------------------+
|1       |(Dinosaur Planet, 2003)                                                            |
|2       |(Isle of Man TT 2004 Review, 2004)                                                 |
|3       |(Character, 1997)                                                                  |
|4       |(Paula Abdul's Get Up & Dance, 1994)                                               |
|5       |(The Rise and Fall of ECW, 2004)                                                   |
|6       |(Sick, 1997)                                                                       |
|7       |(8 Man, 1992)                                                                      |
|8       |(What the #$*! Do We Know!?, 2004)      

In [105]:
#Importando arq. customers_rating.csv

df2= spark.read.options(header=True, inferSchema='True',delimiter=';').csv('/content/drive/MyDrive/Colab Notebooks/customers_rating .csv')

In [106]:
#Vizualizando DF

df2.show()

+-------+------+----------+--------+
|Cust_Id|Rating|      Date|Movie_Id|
+-------+------+----------+--------+
|1488844|   3.0|2005-09-06|       1|
| 822109|   5.0|2005-05-13|       1|
| 885013|   4.0|2005-10-19|       1|
|  30878|   4.0|2005-12-26|       1|
| 823519|   3.0|2004-05-03|       1|
| 893988|   3.0|2005-11-17|       1|
| 124105|   4.0|2004-08-05|       1|
|1248029|   3.0|2004-04-22|       1|
|1842128|   4.0|2004-05-09|       1|
|2238063|   3.0|2005-05-11|       1|
|1503895|   4.0|2005-05-19|       1|
|2207774|   5.0|2005-06-06|       1|
|2590061|   3.0|2004-08-12|       1|
|   2442|   3.0|2004-04-14|       1|
| 543865|   4.0|2004-05-28|       1|
|1209119|   4.0|2004-03-23|       1|
| 804919|   4.0|2004-06-10|       1|
|1086807|   3.0|2004-12-28|       1|
|1711859|   4.0|2005-05-08|       1|
| 372233|   5.0|2005-11-23|       1|
+-------+------+----------+--------+
only showing top 20 rows



In [107]:
#Join Filmes e DF2 prlo Movie_Id

df=df2.join(filmes, df2.Movie_Id == filmes.Movie_Id,'inner')\
      .select(df2.Movie_Id, df2.Rating,df2.Date,filmes.titulo_ano)

In [108]:
#Vizualizando DF

df.show()

+--------+------+----------+--------------------+
|Movie_Id|Rating|      Date|          titulo_ano|
+--------+------+----------+--------------------+
|       1|   3.0|2005-09-06|(Dinosaur Planet,...|
|       1|   5.0|2005-05-13|(Dinosaur Planet,...|
|       1|   4.0|2005-10-19|(Dinosaur Planet,...|
|       1|   4.0|2005-12-26|(Dinosaur Planet,...|
|       1|   3.0|2004-05-03|(Dinosaur Planet,...|
|       1|   3.0|2005-11-17|(Dinosaur Planet,...|
|       1|   4.0|2004-08-05|(Dinosaur Planet,...|
|       1|   3.0|2004-04-22|(Dinosaur Planet,...|
|       1|   4.0|2004-05-09|(Dinosaur Planet,...|
|       1|   3.0|2005-05-11|(Dinosaur Planet,...|
|       1|   4.0|2005-05-19|(Dinosaur Planet,...|
|       1|   5.0|2005-06-06|(Dinosaur Planet,...|
|       1|   3.0|2004-08-12|(Dinosaur Planet,...|
|       1|   3.0|2004-04-14|(Dinosaur Planet,...|
|       1|   4.0|2004-05-28|(Dinosaur Planet,...|
|       1|   4.0|2004-03-23|(Dinosaur Planet,...|
|       1|   4.0|2004-06-10|(Dinosaur Planet,...|


# **1.1. Quantos filmes estão disponíveis no dataset?**

In [109]:
#.count() faz a contagem das celulas na culuna
#.drop_duplicates remove celulas duplicas

df_total=filmes.select('titulo_ano').drop_duplicates().count()
df_total

#saida

4498

# **1.2. Qual é o nome dos 5 filmes com melhor média de avaliação?**

In [110]:
#Cauculo de media da culuna Rating agrupando pelo Movie_Id


df2.groupBy('Movie_Id').agg(f.mean('Rating')).orderBy("avg(Rating)",ascending=False).show(5)

#Saída:

+--------+------------------+
|Movie_Id|       avg(Rating)|
+--------+------------------+
|    3456|4.6709891019450955|
|    3033| 4.586363636363636|
|    2102| 4.581295988606693|
|    4238| 4.554434413170473|
|      13|             4.552|
+--------+------------------+
only showing top 5 rows



In [111]:
#filtro para localizar na tabela filmes os id's com melhor media encontrados acima.

filmes.filter((filmes['Movie_Id']=='3456') |
              (filmes['Movie_Id']=='3033') |
              (filmes['Movie_Id']=='2102') |
              (filmes['Movie_Id']=='4238') |
              (filmes['Movie_Id']=='13')).show(truncate=False)

#Saída:

+--------+-----------------------------------------------------------------------------------+
|Movie_Id|titulo_ano                                                                         |
+--------+-----------------------------------------------------------------------------------+
|13      |(Lord of the Rings: The Return of the King: Extended Edition: Bonus Material, 2003)|
|2102    |(The Simpsons: Season 6, 1994)                                                     |
|3033    |(Ghost in the Shell: Stand Alone Complex: 2nd Gig, 2005)                           |
|3456    |(Lost: Season 1, 2004)                                                             |
|4238    |(Inu-Yasha, 2000)                                                                  |
+--------+-----------------------------------------------------------------------------------+



In [112]:
#No caso de tabelas que receberam um join basta Agrupar pelo Titulo e fazer a media por Rating.

df.groupBy('titulo_ano').agg(f.mean('Rating')).orderBy("avg(Rating)",ascending=False).show(5,truncate=False)

#Saída:

+-----------------------------------------------------------------------------------+------------------+
|titulo_ano                                                                         |avg(Rating)       |
+-----------------------------------------------------------------------------------+------------------+
|(Lost: Season 1, 2004)                                                             |4.6709891019450955|
|(Ghost in the Shell: Stand Alone Complex: 2nd Gig, 2005)                           |4.586363636363636 |
|(The Simpsons: Season 6, 1994)                                                     |4.581295988606693 |
|(Inu-Yasha, 2000)                                                                  |4.554434413170473 |
|(Lord of the Rings: The Return of the King: Extended Edition: Bonus Material, 2003)|4.552             |
+-----------------------------------------------------------------------------------+------------------+
only showing top 5 rows



# **1.3. Quais os 5 anos com menos lançamentos de filmes?**

In [113]:
#criando um df com a coluna titulo_ano separada

filmes2= filmes.withColumn('ano', f.split(filmes['titulo_ano'], ','))\
               .withColumn('titulo_ano', f.col('ano')[0])\
               .withColumn('ano', f.col('ano')[1])
filmes2.show(truncate=False)

#saída:

+--------+----------------------------------------------------------------------------+------+
|Movie_Id|titulo_ano                                                                  |ano   |
+--------+----------------------------------------------------------------------------+------+
|1       |(Dinosaur Planet                                                            | 2003)|
|2       |(Isle of Man TT 2004 Review                                                 | 2004)|
|3       |(Character                                                                  | 1997)|
|4       |(Paula Abdul's Get Up & Dance                                               | 1994)|
|5       |(The Rise and Fall of ECW                                                   | 2004)|
|6       |(Sick                                                                       | 1997)|
|7       |(8 Man                                                                      | 1992)|
|8       |(What the #$*! Do We Know!?             

In [114]:
#contagem de lançamentos agrupados por ano e ordenado do menos para maior.

anos_menos_lanc=filmes2.groupBy('ano').agg(f.count('ano')).orderBy('count(ano)', ascending=True).show(5)

#saída:

+------+----------+
|   ano|count(ano)|
+------+----------+
| 1915)|         1|
| 1922)|         1|
| 1917)|         1|
| 1926)|         1|
| 1918)|         2|
+------+----------+
only showing top 5 rows



# **1.4.Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?**

In [115]:
#Modificando o tipo da coluna Data que estava como string.

data= df.withColumn("Data", df["Date"].cast('Date')).drop('Date')
data.show()



+--------+------+--------------------+----------+
|Movie_Id|Rating|          titulo_ano|      Data|
+--------+------+--------------------+----------+
|       1|   3.0|(Dinosaur Planet,...|2005-09-06|
|       1|   5.0|(Dinosaur Planet,...|2005-05-13|
|       1|   4.0|(Dinosaur Planet,...|2005-10-19|
|       1|   4.0|(Dinosaur Planet,...|2005-12-26|
|       1|   3.0|(Dinosaur Planet,...|2004-05-03|
|       1|   3.0|(Dinosaur Planet,...|2005-11-17|
|       1|   4.0|(Dinosaur Planet,...|2004-08-05|
|       1|   3.0|(Dinosaur Planet,...|2004-04-22|
|       1|   4.0|(Dinosaur Planet,...|2004-05-09|
|       1|   3.0|(Dinosaur Planet,...|2005-05-11|
|       1|   4.0|(Dinosaur Planet,...|2005-05-19|
|       1|   5.0|(Dinosaur Planet,...|2005-06-06|
|       1|   3.0|(Dinosaur Planet,...|2004-08-12|
|       1|   3.0|(Dinosaur Planet,...|2004-04-14|
|       1|   4.0|(Dinosaur Planet,...|2004-05-28|
|       1|   4.0|(Dinosaur Planet,...|2004-03-23|
|       1|   4.0|(Dinosaur Planet,...|2004-06-10|


In [116]:
#vizualizando  

data.printSchema

<bound method DataFrame.printSchema of DataFrame[Movie_Id: int, Rating: double, titulo_ano: string, Data: date]>

In [117]:
#Localizar a Ultima data de Avaliação usando a função .Max()

data_max=data.agg(f.max('Data')).show()

#Saída:

+----------+
| max(Data)|
+----------+
|2005-12-31|
+----------+



In [118]:
#Filtro para localizar filmes avaliados na data encontrada
#media das avaliações dos filmes localizados agrupados por Movie_id e Titulo

ultima_data= data.filter(data.Data=='2005-12-31')
mean_ultima_data= ultima_data.groupBy('Movie_id','titulo_ano').agg(f.mean('Rating').alias('media'))
mean_ultima_data.show()

#Saída:

+--------+--------------------+------------------+
|Movie_id|          titulo_ano|             media|
+--------+--------------------+------------------+
|     426|(Recess: School's...|               4.0|
|     482|       (Frida, 2002)| 4.666666666666667|
|     725|(Woman Thou Art L...|               1.0|
|     763|(The Poseidon Adv...|               5.0|
|     275|      (Evelyn, 2002)|               5.0|
|     338|(The Three Stooge...|               5.0|
|     537|(Here is Greenwoo...|               3.0|
|     457|(Kill Bill: Vol. ...| 3.857142857142857|
|     962|       (I Spy, 2002)|2.6666666666666665|
|      30|(Something's Gott...|              3.68|
|      79| (The Killing, 1956)|               4.0|
|     210|     (Onmyoji, 2001)|               5.0|
|     316|(Futurama: Monste...|               5.0|
|     989|(The Door in the ...|               3.0|
|       8|(What the #$*! Do...|3.6666666666666665|
|     809|(Left Behind II: ...|              3.25|
|     239|(Winnie the Pooh:...|

In [119]:
#Filtro no df de medias para encontrar filmes com avaliação maior ou Igual a 4.7.
#.count() retorna a contagem das celulas. Que totaliza os filmes encontrados acima

total_filmes= mean_ultima_data.filter(mean_ultima_data.media >= 4.7)
total_filmes.count()

#saída:

195


# **1.5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?**

In [120]:
#Filmes encontrados na questao anterior ordenados da menor pra maior media
#exibindo os 10 com a menores notas

total_filmes.orderBy('media',ascending=True).show(10,truncate=False)

+--------+---------------------------------------+-----------------+
|Movie_id|titulo_ano                             |media            |
+--------+---------------------------------------+-----------------+
|3446    |(Spirited Away, 2002)                  |4.714285714285714|
|1800    |(An Evening With Kevin Smith, 2002)    |4.75             |
|2129    |(The Twilight Zone: Vol. 41, 1960)     |4.75             |
|2585    |(Absolutely Fabulous: Series 2, 1994)  |4.75             |
|3551    |(Pete's Dragon, 1977)                  |4.75             |
|4409    |(SpongeBob SquarePants: Season 3, 2002)|4.75             |
|1947    |(Gilmore Girls: Season 3, 2002)        |4.75             |
|1208    |(The Twilight Zone: Vol. 15, 1963)     |4.75             |
|677     |(In the Mood for Love, 2001)           |4.8              |
|4353    |(Curb Your Enthusiasm: Season 3, 2002) |4.8              |
+--------+---------------------------------------+-----------------+
only showing top 10 rows



# **1.6. Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?**

In [121]:
#Contagem das celulas agrupando pelo Cust_Id ordenado do maior para menor

df2.groupBy('Cust_Id').count().orderBy('count',ascending=False).show(5)

+-------+-----+
|Cust_Id|count|
+-------+-----+
| 305344| 4467|
| 387418| 4422|
|2439493| 4195|
|1664010| 4019|
|2118461| 3769|
+-------+-----+
only showing top 5 rows

